In [ ]:
# ==============================================================================
# LANGKAH 0: IMPORT SEMUA LIBRARY YANG DIBUTUHKAN
# ==============================================================================
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import matplotlib.pyplot as plt
import logging
from datetime import datetime

# Menyembunyikan output log yang 'berisik'
logging.getLogger('statsmodels').setLevel(logging.ERROR)


# ==============================================================================
# LANGKAH 1: PENGATURAN VARIABEL UTAMA
# ==============================================================================
# Pilih jenis pangan yang ingin dianalisis
JENIS_PANGAN = 'bawang_merah'
# Tentukan rentang tahun untuk data training
TAHUN_AWAL = 2025
TAHUN_AKHIR = 2025
# Tentukan berapa hari ke depan untuk diprediksi
JUMLAH_HARI_PREDIKSI = 90
# Model yang digunakan
MODEL_PILIHAN = 'SARIMA'


# ==============================================================================
# LANGKAH 2: MEMUAT DAN MEMBERSIHKAN DATA
# ==============================================================================
print("1. Memuat dan mempersiapkan data...")
df = pd.read_csv('/content/tbl_analisis.csv')

# Konversi bulan dan buat kolom datetime 'ds'
bulan_map = {
    'januari': 1, 'februari': 2, 'maret': 3, 'april': 4, 'mei': 5, 'juni': 6,
    'juli': 7, 'agustus': 8, 'september': 9, 'oktober': 10, 'november': 11, 'desember': 12
}
df['bulan'] = df['bulan'].astype(str).str.lower().map(bulan_map)
df['ds'] = pd.to_datetime(df[['tahun', 'bulan', 'tanggal']].rename(columns={'tahun': 'year', 'bulan': 'month', 'tanggal': 'day'}))

# Filter data berdasarkan jenis dan rentang tahun
df_filtered = df[
    (df['jenis'] == JENIS_PANGAN) &
    (df['ds'].dt.year >= TAHUN_AWAL) &
    (df['ds'].dt.year <= TAHUN_AKHIR)
].copy()
df_filtered = df_filtered.rename(columns={'harga': 'y'})
# Untuk SARIMA, kita set tanggal sebagai index
df_filtered = df_filtered.set_index('ds').sort_index()[['y']]


# ==============================================================================
# LANGKAH 3: PENANGANAN OUTLIER
# ==============================================================================
print("\n2. Mendeteksi dan menangani outlier...")
Q1 = df_filtered['y'].quantile(0.25)
Q3 = df_filtered['y'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_count = ((df_filtered['y'] < lower_bound) | (df_filtered['y'] > upper_bound)).sum()
print(f"Ditemukan {outliers_count} outlier.")
df_filtered.loc[(df_filtered['y'] < lower_bound) | (df_filtered['y'] > upper_bound), 'y'] = np.nan
df_filtered['y'] = df_filtered['y'].interpolate(method='linear')
print("Outlier telah ditangani dengan interpolasi.")


# ==============================================================================
# LANGKAH 4 & 5: TRAINING MODEL & MENCARI PARAMETER TERBAIK DENGAN AUTO-ARIMA
# ==============================================================================
print(f"\n3. Mencari parameter {MODEL_PILIHAN} terbaik dan melatih model...")
# auto_arima akan mencari kombinasi (p,d,q)(P,D,Q) terbaik
# m=7 menandakan siklus mingguan (karena data harian)
final_model = auto_arima(df_filtered['y'], 
                         start_p=1, start_q=1,
                         test='adf',       # Uji adf untuk menemukan d
                         max_p=3, max_q=3, # Maksimum p dan q
                         m=7,              # Frekuensi musiman (mingguan)
                         d=None,           # Biarkan model menemukan d
                         seasonal=True,    # Aktifkan SARIMA
                         start_P=0, 
                         D=1, 
                         trace=True,       # Tampilkan proses pencarian
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)    # Gunakan stepwise untuk pencarian lebih cepat

print("\nRingkasan Model SARIMA Terbaik:")
print(final_model.summary())

print("\n4. Membuat prediksi...")
# Membuat prediksi untuk N hari ke depan
forecast, conf_int = final_model.predict(n_periods=JUMLAH_HARI_PREDIKSI, return_conf_int=True)
future_dates = pd.date_range(start=df_filtered.index[-1] + pd.Timedelta(days=1), periods=JUMLAH_HARI_PREDIKSI)
forecast_df = pd.DataFrame(forecast, index=future_dates, columns=['yhat'])
conf_int_df = pd.DataFrame(conf_int, index=future_dates, columns=['yhat_lower', 'yhat_upper'])
plot_df = pd.concat([df_filtered.rename(columns={'y':'yhat'}), forecast_df, conf_int_df], axis=1)


# ==============================================================================
# LANGKAH 6: VISUALISASI HASIL AKHIR
# ==============================================================================
print("5. Menampilkan grafik hasil akhir...")
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(15, 8))

# Judul Dinamis
judul_plot_utama = (
    f"Prediksi Harga '{JENIS_PANGAN.replace('_', ' ').title()}' Selama {JUMLAH_HARI_PREDIKSI} Hari\n"
    f"Model: {MODEL_PILIHAN} {final_model.order}"
)
ax.set_title(judul_plot_utama, size=16)

# Plot data historis
ax.plot(df_filtered.index, df_filtered['y'], 'k-', label='Data Historis')
# Plot data prediksi
ax.plot(forecast_df.index, forecast_df['yhat'], 'b-', label='Prediksi')
# Plot rentang kepercayaan
ax.fill_between(conf_int_df.index,
                conf_int_df['yhat_lower'],
                conf_int_df['yhat_upper'],
                color='blue', alpha=0.2, label='Rentang Kepercayaan')

ax.set_xlabel("Tanggal")
ax.set_ylabel("Harga")
ax.axvline(df_filtered.index.max(), color='red', linestyle='--', lw=2, label='Awal Prediksi')
ax.axvline(datetime.now(), color='green', linestyle=':', lw=2, label='Hari Ini')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

print("\n✅ Proses Selesai!")